In [1]:
# import dependendencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd

In [2]:
df = pd.read_csv('uscities.csv')


In [3]:
cities = list(df['city'])
state_ids = list(df['state_id'])

In [4]:
mod_cities = []
mod_state_ids = []

for city, state_id in zip(cities, state_ids):
    mod_city = city.replace(' ', '-').lower()
    mod_cities.append(mod_city)
    
    mod_state_id = state_id.lower()
    mod_state_ids.append(mod_state_id)
    

In [5]:
selected_columns = df[['lat', 'lng', 'population']]
uscities_df = selected_columns.copy()
uscities_df['state'] = mod_state_ids
uscities_df['city'] = mod_cities
cols = ['state', 'city', 'lat', 'lng', 'population']
uscities_df = uscities_df[cols]


In [6]:
filtered_df = uscities_df.loc[(uscities_df['state'] == 'ny') | (uscities_df['state'] == 'nj') | (uscities_df['state'] == 'pa')]
filtered_df = filtered_df.sort_values(by=['state']).reset_index(drop = True)


In [7]:
states = ['nj', 'ny', 'pa']

state_dfs = []

def create_state_dfs(source_df, states):
   
    for state in states:
        state_df  = source_df.loc[source_df['state'] == state]
        state_df = state_df.sort_values(by=['population'], ascending=False).reset_index(drop = True).head(25)
        state_dfs.append(state_df)
        state_df.empty


create_state_dfs(filtered_df, states) 


# Concatenate the dataframes
top_cities_df = pd.concat(state_dfs, ignore_index=True)


In [8]:
# Open the browser from which we want to extract 
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

url = "https://www.meetup.com/find/us--ny--new-york/outdoors-adventure"
browser.visit(url)
html = browser.html

In [9]:
soup = bs(html, 'html.parser')

In [10]:
# Scrape categories
categories = soup.find_all('div', 'css-1q80wuk')
categories

categories_lst = []

for category in categories:
    for a in category.find_all('a', href=True):
        link_text = a['href']
        category_link = link_text.split('/')
        categories_lst.append(category_link[3])
    

categories_lst

['outdoors-adventure',
 'tech',
 'parents-family',
 'health-wellness',
 'sports-fitness',
 'education',
 'photography',
 'food',
 'writing',
 'language',
 'music',
 'movements',
 'lgbtq',
 'film',
 'games-sci-fi',
 'beliefs',
 'arts-culture',
 'book-clubs',
 'dancing',
 'pets',
 'hobbies-crafts',
 'fashion-beauty',
 'social',
 'career-business']

In [11]:
browser.quit()

In [12]:
# construct urls
category_links = []
for index, row in top_cities_df.iterrows():
    for category in categories_lst:
        cat_url = f"https://www.meetup.com/find/us--{row['state']}--{row['city']}/{category}/?eventType=inPerson&distance=tenMiles"
        category_links.append(cat_url)
        
category_links


['https://www.meetup.com/find/us--nj--trenton/outdoors-adventure/?eventType=inPerson&distance=tenMiles',
 'https://www.meetup.com/find/us--nj--trenton/tech/?eventType=inPerson&distance=tenMiles',
 'https://www.meetup.com/find/us--nj--trenton/parents-family/?eventType=inPerson&distance=tenMiles',
 'https://www.meetup.com/find/us--nj--trenton/health-wellness/?eventType=inPerson&distance=tenMiles',
 'https://www.meetup.com/find/us--nj--trenton/sports-fitness/?eventType=inPerson&distance=tenMiles',
 'https://www.meetup.com/find/us--nj--trenton/education/?eventType=inPerson&distance=tenMiles',
 'https://www.meetup.com/find/us--nj--trenton/photography/?eventType=inPerson&distance=tenMiles',
 'https://www.meetup.com/find/us--nj--trenton/food/?eventType=inPerson&distance=tenMiles',
 'https://www.meetup.com/find/us--nj--trenton/writing/?eventType=inPerson&distance=tenMiles',
 'https://www.meetup.com/find/us--nj--trenton/language/?eventType=inPerson&distance=tenMiles',
 'https://www.meetup.com/f

In [13]:
# category_links = ['https://www.meetup.com/find/us--ny--new-york/outdoors-adventure/?eventType=inPerson&distance=tenMiles']
# 'https://www.meetup.com/find/us--nj--trenton/arts-culture/?eventType=inPerson&distance=tenMiles'
# 'https://www.meetup.com/find/us--ny--new-york/outdoors-adventure/?eventType=inPerson&distance=tenMiles'
events_arr = []
group_names_arr = []
attendees_lst = []
event_links = []
event_dfs = []

for link in category_links:
    
    executable_path = {'executable_path': 'chromedriver.exe'}
    browser = Browser('chrome', **executable_path, headless=False)
      
    browser.visit(link)
    html = browser.html
    
    cat_soup = bs(html, 'html.parser')
    
    # Retrieve state
    state = link.split('--')[1]

    
    # Retrieve city
    city = link.split('--')[2].split('/')[0]

    
#     Retrieve category
    category = link.split('/?')[0].split('/')
    category = category[5]
    

    # Retrieve events
    events = cat_soup.find_all('p', class_='css-1jy1jkx') 
    for event in events:
        events_arr.append(event.text) 

    # Retrieve Group names
    events_descriptions = cat_soup.find_all('p', class_='css-ycqk9')
    for desc in events_descriptions:
        desc_text = desc.text
        desc_text = desc_text.split(':')[1]
        group_names_arr.append(desc_text)

    
    # Retrieve number of attendees
    attendees = cat_soup.find_all('div', class_="css-1kw0tju")
    for attendee_count in attendees:
        attendee_count = attendee_count.text
        attendee_count = attendee_count.split(' ')[0]
        attendees_lst.append(attendee_count)

        
    # Retrieve event link
    events = cat_soup.find_all('div', class_='css-11tad7j')
    for event in events:
        for a in event.find_all('a', href=True):
            link_text = a['href']
            event_links.append(link_text)
            
            
    event_data = {'state':state,
                  'city': city,
                  'category' : category,
                  'event_name': events_arr,
                  'group_name': group_names_arr,
                  'attendees': attendees_lst,
                  'event_link': event_links}
    
    browser.quit()
    
    #Create a dataframe with the scraped data
    df = pd.DataFrame(event_data)
    event_dfs.append(df)
    df.empty
    
    events_arr = []
    group_names_arr = []
    attendees_lst = []
    event_links = []



# print(events_arr)
# print(group_names_arr)   
# print(attendees_lst)
# print(event_links)


In [14]:
# Concatenate all the dataframes from the list

final_meetup_df = pd.concat(event_dfs, ignore_index=True)
final_meetup_df


,state,city,category,event_name,group_name,attendees,event_link
0,nj,trenton,outdoors-adventure,Winter Hike At Crystal Lake,DelawareRiverHikers,14,https://www.meetup.com/delawareriverhikers/eve...
1,nj,trenton,outdoors-adventure,THURSDAY AFTERNOON AEROBIC HIKE,Washington Crossing Park Association of NJ,8,https://www.meetup.com/washingtoncrossingstate...
2,nj,trenton,outdoors-adventure,Scenic Overlook Pokadot Chicken Hike at Baldpa...,Hiking Around Philly,14,https://www.meetup.com/HikingAroundPhilly/even...
3,nj,trenton,tech,Fly-In at Ellisdale Fields,NJ Drone User Group,1,https://www.meetup.com/NJ-Dug/events/275376752
4,nj,trenton,health-wellness,Lifeback's Helping Families Affected By Addiction,Lifeback's Helping Families Affected By Addiction,1,https://www.meetup.com/Lifebacks-Helping-Famil...
...,...,...,...,...,...,...,...
1349,pa,levittown,career-business,Free Hard Networking Group w/ 26 Members - Loo...,Lower Bucks Business Connections,1,https://www.meetup.com/Lower-Bucks-Business-Co...
1350,pa,levittown,career-business,Trenton Real Estate --- hear from talented loc...,Trenton Real Estate Hackers Meetup,2,https://www.meetup.com/Trenton-Real-Estate-Inv...
1351,pa,levittown,career-business,Letip of Newtown,Philly Biz Connects,1,https://www.meetup.com/phillybizconnects/event...
1352,pa,wilkes-barre,sports-fitness,Burn and Tone (for All Fitness Levels),NEPA Performance Workouts for All Fitness Levels,1,https://www.meetup.com/Mountain-Top-Performanc...


In [15]:
# Modify state and city columns in final_meetup_df

final_meetup_df['state'] = final_meetup_df['state'].str.upper()
final_meetup_df['city'] = final_meetup_df['city'].str.title().replace('-', ' ')
final_meetup_df['city'] = final_meetup_df['city'].str.replace('-', ' ')
final_meetup_df

,state,city,category,event_name,group_name,attendees,event_link
0,NJ,Trenton,outdoors-adventure,Winter Hike At Crystal Lake,DelawareRiverHikers,14,https://www.meetup.com/delawareriverhikers/eve...
1,NJ,Trenton,outdoors-adventure,THURSDAY AFTERNOON AEROBIC HIKE,Washington Crossing Park Association of NJ,8,https://www.meetup.com/washingtoncrossingstate...
2,NJ,Trenton,outdoors-adventure,Scenic Overlook Pokadot Chicken Hike at Baldpa...,Hiking Around Philly,14,https://www.meetup.com/HikingAroundPhilly/even...
3,NJ,Trenton,tech,Fly-In at Ellisdale Fields,NJ Drone User Group,1,https://www.meetup.com/NJ-Dug/events/275376752
4,NJ,Trenton,health-wellness,Lifeback's Helping Families Affected By Addiction,Lifeback's Helping Families Affected By Addiction,1,https://www.meetup.com/Lifebacks-Helping-Famil...
...,...,...,...,...,...,...,...
1349,PA,Levittown,career-business,Free Hard Networking Group w/ 26 Members - Loo...,Lower Bucks Business Connections,1,https://www.meetup.com/Lower-Bucks-Business-Co...
1350,PA,Levittown,career-business,Trenton Real Estate --- hear from talented loc...,Trenton Real Estate Hackers Meetup,2,https://www.meetup.com/Trenton-Real-Estate-Inv...
1351,PA,Levittown,career-business,Letip of Newtown,Philly Biz Connects,1,https://www.meetup.com/phillybizconnects/event...
1352,PA,Wilkes Barre,sports-fitness,Burn and Tone (for All Fitness Levels),NEPA Performance Workouts for All Fitness Levels,1,https://www.meetup.com/Mountain-Top-Performanc...


In [16]:
# Modify state and city columns in top_cities_df

top_cities_df['state'] = top_cities_df['state'].str.upper()
top_cities_df['city'] = top_cities_df['city'].str.title()
top_cities_df['city'] = top_cities_df['city'].str.replace('-', ' ')
top_cities_df

,state,city,lat,lng,population
0,NJ,Trenton,40.2236,-74.7641,290716
1,NJ,Newark,40.7245,-74.1725,282011
2,NJ,Jersey City,40.7161,-74.0682,262075
3,NJ,Atlantic City,39.3797,-74.4527,237054
4,NJ,Paterson,40.9147,-74.1624,145233
...,...,...,...,...,...
70,PA,Chambersburg,39.9315,-77.6556,53083
71,PA,Levittown,40.1537,-74.8530,51726
72,PA,Bloomsburg,41.0027,-76.4561,49840
73,PA,Uniontown,39.8994,-79.7244,48137


In [36]:
# Merge the dataframes to include Lat, Lng and Population 
new_df = pd.merge(left=final_meetup_df, right=top_cities_df, how='left', left_on=['state', 'city'], right_on=['state', 'city'])
new_df

,state,city,category,event_name,group_name,attendees,event_link,lat,lng,population
0,NJ,Trenton,outdoors-adventure,Winter Hike At Crystal Lake,DelawareRiverHikers,14,https://www.meetup.com/delawareriverhikers/eve...,40.2236,-74.7641,290716
1,NJ,Trenton,outdoors-adventure,THURSDAY AFTERNOON AEROBIC HIKE,Washington Crossing Park Association of NJ,8,https://www.meetup.com/washingtoncrossingstate...,40.2236,-74.7641,290716
2,NJ,Trenton,outdoors-adventure,Scenic Overlook Pokadot Chicken Hike at Baldpa...,Hiking Around Philly,14,https://www.meetup.com/HikingAroundPhilly/even...,40.2236,-74.7641,290716
3,NJ,Trenton,tech,Fly-In at Ellisdale Fields,NJ Drone User Group,1,https://www.meetup.com/NJ-Dug/events/275376752,40.2236,-74.7641,290716
4,NJ,Trenton,health-wellness,Lifeback's Helping Families Affected By Addiction,Lifeback's Helping Families Affected By Addiction,1,https://www.meetup.com/Lifebacks-Helping-Famil...,40.2236,-74.7641,290716
...,...,...,...,...,...,...,...,...,...,...
1349,PA,Levittown,career-business,Free Hard Networking Group w/ 26 Members - Loo...,Lower Bucks Business Connections,1,https://www.meetup.com/Lower-Bucks-Business-Co...,40.1537,-74.8530,51726
1350,PA,Levittown,career-business,Trenton Real Estate --- hear from talented loc...,Trenton Real Estate Hackers Meetup,2,https://www.meetup.com/Trenton-Real-Estate-Inv...,40.1537,-74.8530,51726
1351,PA,Levittown,career-business,Letip of Newtown,Philly Biz Connects,1,https://www.meetup.com/phillybizconnects/event...,40.1537,-74.8530,51726
1352,PA,Wilkes Barre,sports-fitness,Burn and Tone (for All Fitness Levels),NEPA Performance Workouts for All Fitness Levels,1,https://www.meetup.com/Mountain-Top-Performanc...,41.2468,-75.8759,40766


In [37]:
new_df['category'].unique()

array(['outdoors-adventure', 'tech', 'health-wellness', 'food', 'music',
       'games-sci-fi', 'beliefs', 'arts-culture', 'career-business',
       'parents-family', 'education', 'photography', 'writing',
       'movements', 'lgbtq', 'film', 'hobbies-crafts', 'book-clubs',
       'pets', 'social', 'dancing', 'sports-fitness', 'language'],
      dtype=object)

In [38]:
new_df.loc[new_df['category']=='outdoors-adventure', 'category'] =  'Outdoors & Adventure'
new_df.loc[new_df['category']=='tech', 'category'] =  'Tech'
new_df.loc[new_df['category']=='health-wellness', 'category'] =  'Health & Wellness'
new_df.loc[new_df['category']=='food', 'category'] =  'Food & Drink'
new_df.loc[new_df['category']=='music', 'category'] =  'Music'
new_df.loc[new_df['category']=='games-sci-fi', 'category'] =  'Sci-Fi & Games'
new_df.loc[new_df['category']=='beliefs', 'category'] =  'Beliefs'
new_df.loc[new_df['category']=='arts-cultures', 'category'] =  'Arts'
new_df.loc[new_df['category']=='career-business', 'category'] =  'Career & Business'
new_df.loc[new_df['category']=='parents-family', 'category'] =  'Family'
new_df.loc[new_df['category']=='education', 'category'] =  'Learning'
new_df.loc[new_df['category']=='writing', 'category'] =  'Writing'
new_df.loc[new_df['category']=='photography', 'category'] =  'Photography'
new_df.loc[new_df['category']=='movements', 'category'] =  'Movements'
new_df.loc[new_df['category']=='lgbtq', 'category'] =  'LGBTQ'
new_df.loc[new_df['category']=='film', 'category'] =  'Film'
new_df.loc[new_df['category']=='hobbies-crafts', 'category'] =  'Hobbies & Crafts'
new_df.loc[new_df['category']=='book-clubs', 'category'] =  'Book Clubs'
new_df.loc[new_df['category']=='pets', 'category'] =  'Pets'
new_df.loc[new_df['category']=='social', 'category'] =  'Social'
new_df.loc[new_df['category']=='dancing', 'category'] =  'Dance'
new_df.loc[new_df['category']=='sports-fitness', 'category'] =  'Sports & Fitness'
new_df.loc[new_df['category']=='language', 'category'] =  'Language & Culture'
new_df.loc[new_df['category']=='arts-culture', 'category'] =  'Arts'
new_df

,state,city,category,event_name,group_name,attendees,event_link,lat,lng,population
0,NJ,Trenton,Outdoors & Adventure,Winter Hike At Crystal Lake,DelawareRiverHikers,14,https://www.meetup.com/delawareriverhikers/eve...,40.2236,-74.7641,290716
1,NJ,Trenton,Outdoors & Adventure,THURSDAY AFTERNOON AEROBIC HIKE,Washington Crossing Park Association of NJ,8,https://www.meetup.com/washingtoncrossingstate...,40.2236,-74.7641,290716
2,NJ,Trenton,Outdoors & Adventure,Scenic Overlook Pokadot Chicken Hike at Baldpa...,Hiking Around Philly,14,https://www.meetup.com/HikingAroundPhilly/even...,40.2236,-74.7641,290716
3,NJ,Trenton,Tech,Fly-In at Ellisdale Fields,NJ Drone User Group,1,https://www.meetup.com/NJ-Dug/events/275376752,40.2236,-74.7641,290716
4,NJ,Trenton,Health & Wellness,Lifeback's Helping Families Affected By Addiction,Lifeback's Helping Families Affected By Addiction,1,https://www.meetup.com/Lifebacks-Helping-Famil...,40.2236,-74.7641,290716
...,...,...,...,...,...,...,...,...,...,...
1349,PA,Levittown,Career & Business,Free Hard Networking Group w/ 26 Members - Loo...,Lower Bucks Business Connections,1,https://www.meetup.com/Lower-Bucks-Business-Co...,40.1537,-74.8530,51726
1350,PA,Levittown,Career & Business,Trenton Real Estate --- hear from talented loc...,Trenton Real Estate Hackers Meetup,2,https://www.meetup.com/Trenton-Real-Estate-Inv...,40.1537,-74.8530,51726
1351,PA,Levittown,Career & Business,Letip of Newtown,Philly Biz Connects,1,https://www.meetup.com/phillybizconnects/event...,40.1537,-74.8530,51726
1352,PA,Wilkes Barre,Sports & Fitness,Burn and Tone (for All Fitness Levels),NEPA Performance Workouts for All Fitness Levels,1,https://www.meetup.com/Mountain-Top-Performanc...,41.2468,-75.8759,40766


In [34]:
# Export the dataframe to csv
new_df.to_csv('Final_data.csv',index=False)